### Performing Dilation to filter noise

In [32]:
import numpy as np
import cv2

def dilute1(img):
    rt = np.zeros([img.shape[0],img.shape[1]]).astype(int)
    yt = img
    for i in range(2,img.shape[0]-1):
        for j in range(2,img.shape[1]-1):
            if(yt[i][j] == 255):
                rt[i][j] = 255
                rt[i-1][j] = 255
                rt[i+1][j] = 255
                rt[i][j-1] = 255
                rt[i][j+1] = 255
    return rt

def dilute(img):
    rt = np.zeros([img.shape[0],img.shape[1]]).astype(int)
    yt = img
    for i in range(2,img.shape[0]-1):
        pre = img[i-1]
        cur = img[i]
        aft = img[i+1]
        for j in range(2,img.shape[1]-1):
            if(pre[j-1] == 255 and pre[j] == 255 and pre[j+1] == 255):
                if(aft[j-1] == 255 and aft[j] == 255 and aft[j+1] == 255):
                    if(cur[j-1] == 255 and cur[j+1] == 255):
                        rt[i][j] = 255
                    else:
                        rt[i][j] = 0
    return rt
                

img = cv2.imread('noise.jpg',0)
new_img = img
dil = dilute(new_img)
dil1 = dilute1(dil)
dil2 = dilute1(dil1)
n_bound1 = dil1 - dil
n_bound2 = dil2 - dil1
#cv2.imwrite("res_noise5.jpg",dil)
#cv2.imwrite("res_noise_ddil.jpg",dil1)
cv2.imwrite("res_bound1.jpg",n_bound2)

True

### Performing Erosion

In [33]:
import numpy as np
import cv2

def erode1(img):
    rt = np.ones([img.shape[0],img.shape[1]]).astype(int)
    yt = img
    for i in range(2,img.shape[0]-1):
        for j in range(2,img.shape[1]-1):
            if(yt[i][j] == 0):
                rt[i][j] = 0
                rt[i-1][j] = 0
                rt[i+1][j] = 0
                rt[i][j-1] = 0
                rt[i][j+1] = 0
    rt = rt*255
    return rt

def erode(img):
    rt = np.zeros([img.shape[0],img.shape[1]]).astype(int)
    yt = img
    for i in range(2,img.shape[0]-1):
        pre = img[i-1]
        cur = img[i]
        aft = img[i+1]
        for j in range(2,img.shape[1]-1):
            if(pre[j-1] == 0 and pre[j] == 0 and pre[j+1] == 0):
                if(aft[j-1] == 0 and aft[j] == 0 and aft[j+1] == 0):
                    if(cur[j-1] == 0 and cur[j+1] == 0):
                        rt[i][j] = 0
                    else:
                        rt[i][j] = 255
    return rt
                

img = cv2.imread('noise.jpg',0)
new_img = img
ero = erode1(new_img)
close = dilute1(ero)
close = dilute1(close)
black = np.zeros([img.shape[0],img.shape[1]]).astype(int)
bound1 = img - ero
bound2 = dil2 - dil1
ero1 = erode1(ero)
#cv2.imwrite("res_noise4.jpg",ero)
#cv2.imwrite("res_noise_boundary.jpg",bound1)
cv2.imwrite("res_bound2.jpg",bound2)
#cv2.imwrite("res_noise_close.jpg",close)

True

### Algorithm : Opening followed by Closing

In [34]:
img = cv2.imread('noise.jpg',0)
dil = dilute(img)
dil1 = dilute1(dil)
dil2 = dilute1(dil1)
ero = erode1(dil2)
close = dilute1(ero)
close = dilute1(close)
cv2.imwrite('res_noise1.jpg',close)

True

### Algorithm : Closing followed by Opening

In [35]:
ero = erode1(dil2)
close = dilute1(ero)
close = dilute1(close)
dil = dilute(close)
dil1 = dilute1(dil)
dil2 = dilute1(dil1)
ero = erode1(dil2)
cv2.imwrite('res_noise2.jpg',ero)

True

### Point Detection

In [1]:
def normal(black):
    x=y=0
    new_black = np.zeros([black.shape[0],black.shape[1]])
    maxi = np.max(black)
    print(new_black.shape)
    for i in range(0,black.shape[0]):
        for j in range(0,black.shape[1]):
            new_black[i][j] = (black[i][j]/maxi)*255
            if(new_black[i][j] == 255):
                new_black[i][j] = 255
                x=i
                y=j
    return new_black,x,y
def detect_point(img,kernel):
    black = np.zeros([img.shape[0],img.shape[1]])
    for i in range(2,img.shape[0]-1):
        for j in range(2,img.shape[1]-1):
            R=S=T=bound=mid=0
            R = kernel[0][0] *img[i-1][j-1]+kernel[0][1]*img[i-1][j]+kernel[0][2]*img[i-1][j+1]
            S = kernel[1][0] *img[i][j-1]+kernel[1][2]*img[i][j+1]
            T = kernel[2][0] *img[i+1][j-1]+kernel[2][1]*img[i+1][j]+kernel[2][2]*img[i+1][j+1]
            bound = R+S+T
            mid = kernel[1][1]*img[i][j]
            #print(abs(bound-mid))
            #if(abs(bound-mid)>=thres):
                #black[i][j] = 255
            black[i][j] = abs(bound+mid)
    return black            

In [10]:
import cv2
import numpy as np
kernel = ([-1,-1,-1],[-1,8,-1],[-1,-1,-1])
thres = 3800
img = cv2.imread('blade.jpg',0)
black = detect_point(img,kernel)
#cv2.imwrite('black.jpg',black)
detected,x,y = normal(black)
font                   = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
topLeftCornerOfText    = (y+5,x+10)
fontScale              = 0.4
fontColor              = (255,0,0)
lineType               = 1
text1                  = (str(y),str(x))
cv2.putText(detected,str(text1), topLeftCornerOfText, font, fontScale,fontColor,lineType)
cv2.imwrite('detected1.jpg',detected)

(766, 571)


True

### Heuristic Threholding and Image segmentation

In [7]:
import cv2
import numpy as np

def draw_rect(hist):
    x1=z=hist.shape[1]
    q=x2=0
    for i in range(hist.shape[0]):
        if (np.max(hist[i])) !=0 :
            x1 = np.argmax(hist[i])
            if x1<z:
                z = x1
        temp = np.flip(hist[i])
        if (np.max(temp)) != 0:
            x2 = len(hist[i])-np.argmax(temp)-1
            if x2>q:
                q=x2
    hist_trans = np.transpose(hist)
    y1=s=hist_trans.shape[1]
    b=y2=0
    for i in range(hist_trans.shape[0]):
        if (np.max(hist_trans[i])) !=0 :
            y1 = np.argmax(hist_trans[i])
            if y1<s:
                s = y1
        temp = np.flip(hist_trans[i])
        if (np.max(temp)) != 0:
            y2 = len(hist_trans[i])-np.argmax(temp)-1
            if y2>b:
                b=y2
    print(s,b)
    cv2.rectangle(hist,(z,s),(q,b),(255,0,0),1)
    font                   = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX
    bottomLeftCornerOfText = (z-68,b+10)
    topLeftCornerOfText = (z-68,s+10)
    bottomRightCornerOfText = (q-68,b+10)
    topRightCornerOfText = (q-68,s+10)
    fontScale              = 0.4
    fontColor              = (255,0,0)
    lineType               = 1
    text1                   = (str(z),str(s))
    text2                   = (str(z),str(b))
    text3                   = (str(q),str(s))
    text4                   = (str(q),str(b))
    cv2.putText(hist,str(text1), topLeftCornerOfText, font, fontScale,fontColor,lineType)
    cv2.putText(hist,str(text2), bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    cv2.putText(hist,str(text3), topRightCornerOfText, font, fontScale,fontColor,lineType)
    cv2.putText(hist,str(text4), bottomRightCornerOfText, font, fontScale,fontColor,lineType)
    cv2.imwrite("histo1.jpg",hist)

def avg_inten(img,thres):
    m1=m2=g1=g2=c1=c2=0
    histo = np.zeros([img.shape[0],img.shape[1]])
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(img[i][j] > thres):
                m1 = m1 + img[i][j]
                c1 += 1
                histo[i][j] = 255
            else:
                m2 += img[i][j]
                c2 += 1
    g1 = m1/c1
    g2 = m2/c2
    new_thres = (g1+g2)/2
    print(new_thres)
    #print("m1               m2              c1               c2                      g1                               g2              thrs")
    #print("{}       {}          {}          {}           {}                   {}          {}".format(m1,m2,c1,c2,g1,g2,new_thres))
    #print(" ")
    cv2.imwrite("histo.jpg",histo)
    return histo,new_thres

In [10]:
img = cv2.imread('segment.jpg',0)
pre = 0
init_thres = 200
f = 0
while abs(pre - init_thres) > 50:
    print(init_thres)
    histo,new_thres = avg_inten(img,init_thres)
    pre = init_thres
    init_thres = new_thres
    #print(init_thres)
    #f = f+1
hist = cv2.imread('histo.jpg',0)
draw_rect(hist)

200
150.02494476888927
22 284


### Houghs' Transform

In [636]:
import cv2
import numpy as np
import math

img = cv2.imread('hough.jpg')
edges_h = img[:,:,0]
edges = img[:,:,2]
kernel1 = ([-1,-2,-1],[0,0,0],[1,2,1])
kernel2 = ([1,0,-1],[2,0,-2],[1,0,-1]) 
sob_vert_line = np.zeros([edges.shape[0],edges.shape[1]])
sob_hor_line = np.zeros([edges.shape[0],edges.shape[1]])
for i in range(2,edges_h.shape[0]-1):
    for j in range(2,edges_h.shape[1]-1):
        #verticle line
        R=S=T=bound=mid=0
        R = kernel1[0][0] *edges[i-1][j-1]+kernel1[0][1]*edges[i-1][j]+kernel1[0][2]*edges[i-1][j+1]
        S = kernel1[1][0] *edges[i][j-1]+kernel1[1][1]*edges[i][j]+kernel1[1][2]*edges[i][j+1]
        T = kernel1[2][0] *edges[i+1][j-1]+kernel1[2][1]*edges[i+1][j]+kernel1[2][2]*edges[i+1][j+1]
        bound = R+S+T
        if(bound > 80):
            sob_vert_line[i][j] = bound
        #hori line
        R=S=T=bound=mid=0
        R = kernel2[0][0] *edges_h[i-1][j-1]+kernel2[0][1]*edges_h[i-1][j]+kernel2[0][2]*edges_h[i-1][j+1]
        S = kernel2[1][0] *edges_h[i][j-1]+kernel2[1][1]*edges_h[i][j]+kernel2[1][2]*edges_h[i][j+1]
        T = kernel2[2][0] *edges_h[i+1][j-1]+kernel2[2][1]*edges_h[i+1][j]+kernel2[2][2]*edges_h[i+1][j+1]
        bound = R+S+T
        if(bound > 200):
            sob_hor_line[i][j] = bound
cv2.imwrite('1sob.jpg',sob_vert_line)
cv2.imwrite('2sob.jpg',sob_hor_line)

True

In [519]:
edges = sob_vert_line
edges_h =sob_hor_line
kernel1 = ([-1,2,-1],[-1,2,-1],[-1,2,-1])
kernel2 = ([-1,-1,-1],[2,2,2],[-1,-1,-1])
kernel3 = ([2,-1,-1],[-1,2,-1],[-1,-1,2])
kernel4 = ([-1,-1,2],[-1,2,-1],[2,-1,-1])
vert_line = np.zeros([edges.shape[0],edges.shape[1]])
hori_line = np.zeros([edges.shape[0],edges.shape[1]])
ang1_line = np.zeros([edges.shape[0],edges.shape[1]])
ang2_line = np.zeros([edges.shape[0],edges.shape[1]])
for i in range(2,edges.shape[0]-1):
    for j in range(2,edges.shape[1]-1):
        #verticle line
        R=S=T=bound=mid=0
        R = kernel1[0][0] *edges_h[i-1][j-1]+kernel1[0][1]*edges_h[i-1][j]+kernel1[0][2]*edges_h[i-1][j+1]
        S = kernel1[1][0] *edges_h[i][j-1]+kernel1[1][1]*edges_h[i][j]+kernel1[1][2]*edges_h[i][j+1]
        T = kernel1[2][0] *edges_h[i+1][j-1]+kernel1[2][1]*edges_h[i+1][j]+kernel1[2][2]*edges_h[i+1][j+1]
        bound = R+S+T
        vert_line[i][j] = bound
        #horizontal line
        R=S=T=bound=mid=0
        R = kernel2[0][0] *edges[i-1][j-1]+kernel2[0][1]*edges[i-1][j]+kernel2[0][2]*edges[i-1][j+1]
        S = kernel2[1][0] *edges[i][j-1]+kernel2[1][1]*edges[i][j]+kernel2[1][2]*edges[i][j+1]
        T = kernel2[2][0] *edges[i+1][j-1]+kernel2[2][1]*edges[i+1][j]+kernel2[2][2]*edges[i+1][j+1]
        bound = R+S+T
        hori_line[i][j] = bound
        #lines at angle +45
        R=S=T=bound=mid=0
        R = kernel3[0][0] *edges[i-1][j-1]+kernel3[0][1]*edges[i-1][j]+kernel3[0][2]*edges[i-1][j+1]
        S = kernel3[1][0] *edges[i][j-1]+kernel3[1][1]*edges[i][j]+kernel3[1][2]*edges[i][j+1]
        T = kernel3[2][0] *edges[i+1][j-1]+kernel3[2][1]*edges[i+1][j]+kernel3[2][2]*edges[i+1][j+1]
        bound = R+S+T
        ang1_line[i][j] = bound
        #lines at angle -45
        R=S=T=bound=mid=0
        R = kernel4[0][0] *edges[i-1][j-1]+kernel4[0][1]*edges[i-1][j]+kernel4[0][2]*edges[i-1][j+1]
        S = kernel4[1][0] *edges[i][j-1]+kernel4[1][1]*edges[i][j]+kernel4[1][2]*edges[i][j+1]
        T = kernel4[2][0] *edges[i+1][j-1]+kernel4[2][1]*edges[i+1][j]+kernel4[2][2]*edges[i+1][j+1]
        bound = R+S+T
        ang2_line[i][j] = bound


True

In [594]:
def darker(img):
    matrix = np.zeros([img.shape[0],img.shape[1]])
    for i in range(0,img.shape[0]):
        for j in range(0,img.shape[1]):
            if(img[i][j] > 150):
                matrix[i][j] = img[i][j]
    return matrix

def calc_diag(img):
    x1 = img.shape[0]
    y1 = img.shape[1]
    diag = math.sqrt((x1*x1)+(y1*y1))
    return math.ceil(diag)

def huff(slants,diag):
    rho = 0
    matrix = np.zeros([1800,181])
    #for s in range(-90,90):
    for i in range(0,slants.shape[0]):
        for j in range(0,slants.shape[1]):
            for s in range(-90,90):
                if(slants[i][j]>150):
                    rho = math.floor(j* math.cos(math.radians(s)) + i* math.sin(math.radians(s)))
                    matrix[diag-rho][90+s] += 1
    return matrix
    

In [622]:
def highlight(sin_huff):
    mk = np.zeros([sin_huff.shape[0],sin_huff.shape[1]])
    ro = []
    col =[]
    high = np.max(sin_huff)
    print(high)
    sin_h = sin_huff/high*255
    for i in range(0,sin_huff.shape[0]):
        for j in range(0,sin_huff.shape[1]):
            if(sin_huff[i][j]<140):
                mk[i][j] = sin_huff[i][j]
                ro.append(i)
                col.append(j)
            else:
                mk[i][j] = 0
    return mk,ro,col

In [623]:
def draw_lines(mk,ro,theta,diag):
    y1=0
    x1 = (diag-ro)-y1*math.sin(theta-90)/np.cos(np.radians(theta-90))
    y2= mk.shape[1]
    x2 = (diag-ro)-y2*math.sin(theta-90)/np.cos(np.radians(theta-90))
    return x1,x2

In [624]:

diag = calc_diag(vert_line)
#slants = darker(vert_line)
sin_huff = huff(slants,diag)

diag = calc_diag(sob_vert_line)
#slants = darker(sob_vert_line)
sin_huff = huff(sob_vert_line,diag)

In [625]:
#cv2.imwrite('manju2.jpg',slants)
cv2.imwrite('sine1.jpg',sin_huff)
#cv2.imwrite('sine2.jpg',sin_huff)

True

In [626]:
mk,ro,col = highlight(sin_huff)
img = cv2.imread('hough.jpg')

55.0


True

In [618]:
for i in range(len(ro)):
    x1,x2 = draw_lines(mk,ro[i],col[i],diag)
    line = cv2.line(img,(x1.astype(int),0),(x2.astype(int),666),(0,255,0))

In [619]:
cv2.imwrite('red.jpg',line)
#cv2.imwrite('blue.jpg',line)

True

In [602]:
diag

820

### Coin detection

In [644]:
img = cv2.imread('hough.jpg')
cir_img = img.copy()
cir = img[:,:,1]
cir_edge = np.zeros([cir.shape[0],cir.shape[1]])
sobel = ([1,0,-1],[2,0,-2],[1,0,-1])
for i in range(2,cir.shape[0]-1):
    for j in range(2,cir.shape[1]-1):
        R=S=T=bound=mid=0
        R = sobel[0][0] *cir[i-1][j-1]+sobel[0][1]*cir[i-1][j]+sobel[0][2]*cir[i-1][j+1]
        S = sobel[1][0] *cir[i][j-1]+sobel[1][1]*cir[i][j]+sobel[1][2]*cir[i][j+1]
        T = sobel[2][0] *cir[i+1][j-1]+sobel[2][1]*cir[i+1][j]+sobel[2][2]*cir[i+1][j+1]
        bound = R+S+T
        if(bound > 80):
            cir_edge[i][j] = bound
#cv2.imwrite('cir_edge.jpg',cir_edge)       
        
#implement the formulas for a and b
r = 24
x,y = sob_vert_line.shape
accum = np.zeros([1333,1000])
for i in range(x):
    for j in range(y):
        if(sob_vert_line[i][j]>500):
            for theta in (0,360):
                a = math.floor(i - r*math.cos(np.radians(theta)))
                b =  math.floor(j - r*math.sin(np.radians(theta)))
                accum[a][b] += 1
cv2.imwrite('huff_circles.jpg',accum)
        
p = []   #a
q = []  #b
count =0
for i in range(0,accum.shape[0]):
    for j in range(0,accum.shape[1]):
        if(accum[i][j]>150):
            p.append(j)
            q.append(i)
            
for i in range(len(p)):
    cv2.circle(cir_img,center = (p[i],q[i]),radius = 24,color=(0,0,255),thickness =1)
cv2.imwrite('coin.jpg',cir_img)

True

In [645]:
cv2.imwrite('cir_edge.jpg',cir_edge)  

True

In [643]:
np.max(sob_vert_line)

718.0